In [1]:
!git clone https://github.com/caltamiranda/MCE-ROI-V2.git

Cloning into 'MCE-ROI-V2'...
remote: Enumerating objects: 1945, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 1945 (delta 0), reused 0 (delta 0), pack-reused 1943 (from 3)
Receiving objects: 100% (1945/1945), 53.86 MiB | 29.03 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import sys
sys.path.append("/kaggle/working/MCE-ROI-V2/rf_pipeline")
sys.path.append("/kaggle/working/MCE-ROI-V2")

In [3]:
from rf_pipeline.models.hybrid_classifier import HybridRFClassifier


In [4]:
import os
import sys
import importlib

# 1. Definimos el contenido que necesita el repositorio
config_content = """
import os

# --- Parámetros de Señal ---
FS = 10_000_000      
CENTER_FREQ = 0      

# --- Parámetros STFT / Preprocesamiento ---
NPERSEG = 32
NOVERLAP = 16
NFFT = 32            
IMG_SIZE = 32        

# --- Entrenamiento ---
BATCH_SIZE = 32
EPOCHS = 50
LR = 1e-3
DEVICE = "cuda"

# --- Rutas Dummy (se sobrescriben en el notebook, pero deben existir aquí) ---
TRAIN_H5 = ""
VAL_H5 = ""
"""

# 2. Escribimos el archivo config.py en la ruta donde el repo lo busca
repo_path = "/kaggle/working/MCE-ROI-V2"
config_path = os.path.join(repo_path, "config.py")

with open(config_path, "w") as f:
    f.write(config_content)
print(f"Archivo de configuración creado en: {config_path}")

# 3. Forzamos la recarga del módulo config para que Python vea los cambios
# (Esto es necesario si ya intentaste correr la celda anterior y falló)
try:
    import config
    importlib.reload(config)
    print("Módulo config recargado exitosamente.")
except ImportError:
    print("El módulo config se cargará automáticamente en el siguiente paso.")

# 4. Ahora sí importamos las librerías del repo
from core.data_loader import H5HybridDetectionDataset
# Forzamos recarga de data_loader por si acaso quedó con la referencia rota
import core.data_loader
importlib.reload(core.data_loader)
from core.data_loader import H5HybridDetectionDataset

print("Corrección aplicada. Ahora puedes ejecutar la celda de Datasets.")

Archivo de configuración creado en: /kaggle/working/MCE-ROI-V2/config.py
Módulo config recargado exitosamente.
Corrección aplicada. Ahora puedes ejecutar la celda de Datasets.


In [5]:
import torch

class CFG:
    # --- Rutas Exactas ---
    # Asumo que dentro de las carpetas hay un archivo llamado 'data.h5'
    # Si el archivo tiene otro nombre (ej: 'train.h5'), cámbialo al final de la ruta.
    TRAIN_H5 = "/kaggle/input/rf-benchmark-tiny/raw_iq_hdf5/train/data.h5"
    VAL_H5   = "/kaggle/input/rf-benchmark-tiny/raw_iq_hdf5/val/data.h5" # (Ajusta si la ruta de val es distinta)
    TEST_H5  = "/kaggle/input/rf-benchmark-tiny/raw_iq_hdf5/test/data.h5"

    OUTPUT_DIR = "/kaggle/working/resultados_finales"
    
    # Parámetros (Deben coincidir con los del config.py creado arriba)
    BATCH_SIZE = 32
    EPOCHS = 50
    LR = 1e-3
    NUM_WORKERS = 2
    
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
print(f"Entrenando en: {CFG.DEVICE}")

Entrenando en: cpu


In [6]:
from torch.utils.data import DataLoader
import core.data_loader
importlib.reload(core.data_loader) # Recarga segura
from core.data_loader import H5HybridDetectionDataset

print("--- Cargando Datasets ---")

# 1. TRAIN (Para entrenar pesos)
ds_train = H5HybridDetectionDataset(CFG.TRAIN_H5, mode="train")

# 2. VAL (Para validar loss/accuracy en cada época y guardar el mejor modelo)
# Usamos mode="train" para que nos devuelva tensores listos para clasificar, no imágenes completas.
ds_val = H5HybridDetectionDataset(CFG.VAL_H5, mode="train")

# 3. TEST (Para evaluación final)
ds_test = H5HybridDetectionDataset(CFG.TEST_H5, mode="train")

# DataLoaders
dl_train = DataLoader(ds_train, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=CFG.NUM_WORKERS, pin_memory=True)
dl_val   = DataLoader(ds_val, batch_size=CFG.BATCH_SIZE*2, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=True)
dl_test  = DataLoader(ds_test, batch_size=CFG.BATCH_SIZE*2, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=True)

print(f"Muestras de Train: {len(ds_train)}")
print(f"Muestras de Val:   {len(ds_val)}")
print(f"Muestras de Test:  {len(ds_test)}")

--- Cargando Datasets ---
Muestras de Train: 700
Muestras de Val:   150
Muestras de Test:  150


In [7]:
from models.hybrid_classifier import HybridRFClassifier
import torch.optim as optim
import torch.nn as nn

model = HybridRFClassifier(
    num_classes=2, 
    feat_dim=32,
    img_in_ch=3,
    cnn_channels=(16, 32, 64),
    mlp_hidden=(64, 64)
).to(CFG.DEVICE)

optimizer = optim.Adam(model.parameters(), lr=CFG.LR)
criterion = nn.CrossEntropyLoss()

In [8]:
import time
import numpy as np
from sklearn.metrics import accuracy_score

best_val_acc = 0.0
history = []

print("=== Inicio del Entrenamiento ===")

for epoch in range(CFG.EPOCHS):
    start = time.time()
    
    # --- TRAIN ---
    model.train()
    train_loss = 0
    preds_t, targets_t = [], []
    
    for x_vis, x_eng, y in dl_train:
        x_vis, x_eng, y = x_vis.to(CFG.DEVICE), x_eng.to(CFG.DEVICE), y.to(CFG.DEVICE)
        
        optimizer.zero_grad()
        logits = model(x_vis, x_eng)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        preds_t.extend(torch.argmax(logits, dim=1).cpu().numpy())
        targets_t.extend(y.cpu().numpy())

    # --- VAL ---
    model.eval()
    val_loss = 0
    preds_v, targets_v = [], []
    
    with torch.no_grad():
        for x_vis, x_eng, y in dl_val:
            x_vis, x_eng, y = x_vis.to(CFG.DEVICE), x_eng.to(CFG.DEVICE), y.to(CFG.DEVICE)
            logits = model(x_vis, x_eng)
            val_loss += criterion(logits, y).item()
            preds_v.extend(torch.argmax(logits, dim=1).cpu().numpy())
            targets_v.extend(y.cpu().numpy())

    # Métricas
    t_acc = accuracy_score(targets_t, preds_t)
    v_acc = accuracy_score(targets_v, preds_v)
    epoch_time = time.time() - start
    
    print(f"Epoch {epoch+1}/{CFG.EPOCHS} [{epoch_time:.1f}s] | "
          f"Train Acc: {t_acc:.2%} Loss: {train_loss/len(dl_train):.4f} | "
          f"Val Acc: {v_acc:.2%} Loss: {val_loss/len(dl_val):.4f}")
    
    # Guardar el mejor
    if v_acc > best_val_acc:
        best_val_acc = v_acc
        torch.save(model.state_dict(), os.path.join(CFG.OUTPUT_DIR, "best_model.pt"))

print(f"\nEntrenamiento finalizado. Mejor Val Acc: {best_val_acc:.2%}")

=== Inicio del Entrenamiento ===


/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)


Epoch 1/50 [17.6s] | Train Acc: 91.29% Loss: 0.2406 | Val Acc: 98.00% Loss: 0.1002


/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)


Epoch 2/50 [16.7s] | Train Acc: 94.71% Loss: 0.1286 | Val Acc: 98.67% Loss: 0.0378


...

Epoch 48/50 [16.6s] | Train Acc: 99.43% Loss: 0.0131 | Val Acc: 100.00% Loss: 0.0022


/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)


Epoch 49/50 [16.1s] | Train Acc: 99.43% Loss: 0.0158 | Val Acc: 100.00% Loss: 0.0010


/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)


Epoch 50/50 [16.3s] | Train Acc: 99.43% Loss: 0.0108 | Val Acc: 100.00% Loss: 0.0045

Entrenamiento finalizado. Mejor Val Acc: 100.00%


In [9]:
from models.hybrid_classifier import HybridRFClassifier
import torch
import os
from sklearn.metrics import accuracy_score

# 1. Instanciar el modelo EXACTAMENTE igual que en el entrenamiento
# IMPORTANTE: img_in_ch=3 es la clave para solucionar tu error
model = HybridRFClassifier(
    num_classes=2, 
    feat_dim=32,
    img_in_ch=3,            # <--- AQUÍ ESTABA EL ERROR (Antes tenías 1 o por defecto)
    cnn_channels=(16, 32, 64),
    mlp_hidden=(64, 64)
).to(CFG.DEVICE)

# 2. Cargar los pesos del mejor modelo guardado
print("--- Cargando pesos del mejor modelo ---")
checkpoint_path = os.path.join(CFG.OUTPUT_DIR, "best_model.pt")
model.load_state_dict(torch.load(checkpoint_path, map_location=CFG.DEVICE))
model.eval()

# 3. Evaluar en el conjunto de Test
print("--- Evaluando en Test Set ---")
test_preds, test_targets = [], []

with torch.no_grad():
    for x_vis, x_eng, y in dl_test:
        x_vis, x_eng, y = x_vis.to(CFG.DEVICE), x_eng.to(CFG.DEVICE), y.to(CFG.DEVICE)
        
        # Forward
        logits = model(x_vis, x_eng)
        
        # Guardar predicciones
        test_preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        test_targets.extend(y.cpu().numpy())

# 4. Calcular Accuracy Final
final_acc = accuracy_score(test_targets, test_preds)
print(f"\nRESULTADO FINAL EN TEST: {final_acc:.2%}")

--- Cargando pesos del mejor modelo ---
--- Evaluando en Test Set ---


/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)
/kaggle/working/MCE-ROI-V2/rf_pipeline/core/preprocessing.py:24: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = stft(iq_signal, fs=self.fs, nperseg=self.nperseg, noverlap=self.noverlap)



RESULTADO FINAL EN TEST: 99.33%
